<a href="https://colab.research.google.com/github/kheuch2018/Wolof_MFC_PartOfSpeech_Tagger/blob/main/Wolof_MFCT_PartOfSpeech_Tagging.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Wolof Most Frequent Class Tagger

In [ ]:
!pip install pomegranate

import os
os.kill(os.getpid(), 9)

In [1]:
!wget https://raw.githubusercontent.com/UniversalDependencies/UD_Wolof-WTB/master/wo_wtb-ud-train.conllu
!wget https://raw.githubusercontent.com/kheuch2018/NLP_Helper/master/main.py

--2021-04-17 17:39:16--  https://raw.githubusercontent.com/UniversalDependencies/UD_Wolof-WTB/master/wo_wtb-ud-train.conllu
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1622884 (1.5M) [text/plain]
Saving to: ‘wo_wtb-ud-train.conllu’

wo_wtb-ud-train.con 100%[===================>]   1.55M  --.-KB/s    in 0.05s   

2021-04-17 17:39:16 (29.3 MB/s) - ‘wo_wtb-ud-train.conllu’ saved [1622884/1622884]

--2021-04-17 17:39:16--  https://raw.githubusercontent.com/kheuch2018/NLP_Helper/master/main.py
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.111.133, 185.199.110.133, 185.199.108.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.111.133|:443... connected.
HTTP request sent, awaiting response... 200 O

In [3]:
# Jupyter "magic methods" -- only need to be run once per kernel restart
%reload_ext autoreload
%aimport tests
%autoreload 1

In [43]:
import numpy as np
def process_text(path,new_file):
  tags_wolof = set()
  f2 = open(new_file,"w+")
  f3 = open("tags_"+new_file,"w+")
  f = open(path,"r")
  for line in f.readlines():
    if "#" not in line:
      arr = line.split("	")[1:5]
      if len(arr)>0 and arr[0] == arr[1] and arr[2] == arr[3]:
        f2.write(arr[0]+"\t"+arr[2].upper()+"\n")
        tags_wolof.add(arr[2].upper())
      elif len(arr)>0:
        f2.write(arr[0]+"\t"+arr[2].upper()+"\n")
        #f2.write(arr[1]+"\t"+arr[3].upper()+"\n")
        tags_wolof.update([arr[2].upper()]) #,arr[3].upper()
    elif "# newdoc_id" not in line  and "# text " not in line :
      f2.write("\n")
      f2.write(line.replace("# sent_id = ","").replace('"',""))
  for tag in np.unique(list(tags_wolof)):
    f3.write(tag+"\n")

  f2.write("\n")
  

In [44]:
process_text("/content/wo_wtb-ud-train.conllu","processed.txt")

In [45]:
# import python modules -- this cell needs to be run again if you make changes to any of the files
import matplotlib.pyplot as plt
import numpy as np

from IPython.core.display import HTML
from itertools import chain
from collections import Counter, defaultdict
from main import Dataset
from pomegranate import State, HiddenMarkovModel, DiscreteDistribution

In [46]:
len(data.Y)

1188

In [47]:
data = Dataset("/content/processed.txt","/content/tags_processed.txt")

print("There are {} sentences in the corpus.".format(len(data.sentences)))
print("There are {} sentences in the training set.".format(len(data.training_set["sentences"])))
print("There are {} sentences in the testing set.".format(len(data.testing_set["sentences"])))

assert len(data.sentences) == len(data.training_set["sentences"]) + len(data.testing_set["sentences"]), \
       "The number of sentences in the training set + testing set should sum to the number of sentences in the corpus"

There are 1188 sentences in the corpus.
There are 950 sentences in the training set.
There are 238 sentences in the testing set.


In [ ]:
data.sentences_dict

# Building a Most Frequent Class Tagger

In [48]:
def pair_counts(sequences_A, sequences_B):
    """Return a dictionary keyed to each unique value in the first sequence list
    that counts the number of occurrences of the corresponding value from the
    second sequences list.
    
    For example, if sequences_A is tags and sequences_B is the corresponding
    words, then if 1244 sequences contain the word "time" tagged as a NOUN, then
    you should return a dictionary such that pair_counts[NOUN][time] == 1244
    """
    A = ["adj", "noun","noun","noun", "verb"]
    B = ["ma", "grand","mere", "grand", "grand"]

    result={}
    for wordtup,tagtup in zip(sequences_B,sequences_A):
      for word,tag in zip(wordtup,tagtup):
        result.setdefault(tag,{}) #[tag][word] = 0
        result[tag][word] = 0
    for wordtup,tagtup in zip(sequences_B,sequences_A):
      for word,tag in zip(wordtup,tagtup):
        result[tag][word]+=1    
    # TODO: Finish this function!
    #raise NotImplementedError
    return result

# Calculate C(t_i, w_i)
emission_counts = pair_counts(data.Y,data.X)


In [49]:
# Create a lookup table mfc_table where mfc_table[word] contains the tag label most frequently assigned to that word
from collections import namedtuple

FakeState = namedtuple("FakeState", "name")

class MFCTagger:
    # NOTE: You should not need to modify this class or any of its methods
    missing = FakeState(name="<MISSING>")
    
    def __init__(self, table):
        self.table = defaultdict(lambda: MFCTagger.missing)
        self.table.update({word: FakeState(name=tag) for word, tag in table.items()})
        
    def viterbi(self, seq):
        """This method simplifies predictions by matching the Pomegranate viterbi() interface"""
        return 0., list(enumerate(["<start>"] + [self.table[w] for w in seq] + ["<end>"]))


# TODO: calculate the frequency of each tag being assigned to each word (hint: similar, but not
# the same as the emission probabilities) and use it to fill the mfc_table

word_counts = pair_counts(data.training_set["Y"] ,data.training_set["X"])
mfc_table = {} # TODO: YOUR CODE HERE

for word in data.training_set["vocab"]:
  count_word_max = 0
  max_tag = ""
  for tag in word_counts:
    if word in word_counts[tag] and word_counts[tag][word] > count_word_max:
      count_word_max = word_counts[tag][word]
      max_tag = tag
  mfc_table[word] = max_tag

# DO NOT MODIFY BELOW THIS LINE
mfc_model = MFCTagger(mfc_table) # Create a Most Frequent Class tagger instance
assert len(mfc_table) == len(data.training_set["vocab"]), ""
assert all(k in data.training_set["vocab"] for k in mfc_table.keys()), ""
HTML('<div class="alert alert-block alert-success">Your MFC tagger has all the correct words!</div>')


In [50]:
def replace_unknown(sequence):
    """Return a copy of the input sequence where each unknown word is replaced
    by the literal string value 'nan'. Pomegranate will ignore these values
    during computation.
    """
    return [w if w in data.training_set["vocab"] else 'nan' for w in sequence]

def simplify_decoding(X, model):
    """X should be a 1-D sequence of observations for the model to predict"""
    #print("heeeee",replace_unknown(X))
    _, state_path = model.viterbi(replace_unknown(X))
    return [state[1].name for state in state_path[1:-1]]  # do not show the start/end state predictions

In [51]:
list(data.testing_set["keys"])[:3]

['wo_wtb-ud-train_995', 'wo_wtb-ud-train_996', 'wo_wtb-ud-train_997']

# Predicting

In [52]:
for key in list(data.testing_set["keys"])[:3]:
    print("Sentence Key: {}\n".format(key))
    print("Predicted labels:\n-----------------")
    print(simplify_decoding(data.sentences_dict[key]["words"], mfc_model))
    print()
    print("Actual labels:\n--------------")
    print(data.sentences_dict[key]["tags"])
    print("\n")

Sentence Key: wo_wtb-ud-train_995

Predicted labels:
-----------------
['ADP', 'NOUN', 'ADP', 'ADV', 'NOUN', 'DET', 'ADV', 'AUX', 'PUNCT']

Actual labels:
--------------
('ADP', 'ADV', 'ADP', 'NOUN', 'NOUN', 'DET', 'VERB', 'AUX', 'PUNCT')


Sentence Key: wo_wtb-ud-train_996

Predicted labels:
-----------------
['NOUN', 'NOUN', 'ADV', 'ADP', 'NUM', 'ADV', 'PUNCT', '_', 'AUX', 'PRON', 'VERB', 'VERB', 'PUNCT', 'ADV', 'ADV', 'VERB', 'CCONJ', 'PROPN', 'ADV', 'PUNCT', 'DET', 'NUM', 'ADV', 'PRON', 'ADV', 'PRON', 'PUNCT', '_', 'AUX', 'PRON', 'ADV', 'DET', 'NOUN', 'PUNCT']

Actual labels:
--------------
('NOUN', 'PROPN', 'VERB', 'ADP', 'DET', 'NOUN', 'PUNCT', '_', 'AUX', 'PRON', 'VERB', 'VERB', 'PUNCT', 'NUM', 'NOUN', 'VERB', 'ADP', 'PROPN', 'PROPN', 'PUNCT', 'DET', 'NUM', 'NOUN', 'PRON', 'VERB', 'PRON', 'PUNCT', '_', 'AUX', 'PRON', 'VERB', 'DET', 'NOUN', 'PUNCT')


Sentence Key: wo_wtb-ud-train_997

Predicted labels:
-----------------
['CCONJ', 'PRON', 'PRON', 'AUX', 'VERB', 'VERB', 'AUX', 'PU

# Evaluate accuracy of MFC Tagger

In [53]:
def accuracy(X, Y, model):
    """Calculate the prediction accuracy by using the model to decode each sequence
    in the input X and comparing the prediction with the true labels in Y.
    
    The X should be an array whose first dimension is the number of sentences to test,
    and each element of the array should be an iterable of the words in the sequence.
    The arrays X and Y should have the exact same shape.
    
    X = [("See", "Spot", "run"), ("Run", "Spot", "run", "fast"), ...]
    Y = [(), (), ...]
    """
    correct = total_predictions = 0
    for observations, actual_tags in zip(X, Y):
        
        # The model.viterbi call in simplify_decoding will return None if the HMM
        # raises an error (for example, if a test sentence contains a word that
        # is out of vocabulary for the training set). Any exception counts the
        # full sentence as an error (which makes this a conservative estimate).
        try:
            most_likely_tags = simplify_decoding(observations, model)
            correct += sum(p == t for p, t in zip(most_likely_tags, actual_tags))
        except:
            pass
        total_predictions += len(observations)
    return correct / total_predictions

In [54]:
mfc_training_acc = accuracy(data.training_set["X"], data.training_set["Y"], mfc_model)
print("training accuracy mfc_model: {:.2f}%".format(100 * mfc_training_acc))

mfc_testing_acc = accuracy(data.testing_set["X"], data.testing_set["Y"], mfc_model)
print("testing accuracy mfc_model: {:.2f}%".format(100 * mfc_testing_acc))

# assert mfc_training_acc >= 0.955, "Uh oh. Your MFC accuracy on the training set doesn't look right."
# assert mfc_testing_acc >= 0.925, "Uh oh. Your MFC accuracy on the testing set doesn't look right."
# HTML('<div class="alert alert-block alert-success">Your MFC tagger accuracy looks correct!</div>')

training accuracy mfc_model: 92.17%
testing accuracy mfc_model: 74.66%
